In [1]:
from glob import glob

for g in glob('../data/*.pdf'):
    print(g)

../data\2040_seoul_plan.pdf
../data\OneNYC_2050_Strategic_Plan.pdf


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def read_pdf_and_split_text(pdf_path, chunk_size=1000, chunk_overlap=100):
    """
    주어진 pdf 파일을 읽고 텍스트를 분할합니다.
    매개변수:
        pdf_path (str): PDF 파일의 경로.
        chunk_size (int, 선택적): 각 텍스트 청크의 크기, 기본값은 1000입니다.
        chunk_overlap (int, 선택적): 청크 간의 중첩 크기, 기본값은 100입니다.
    반환값:
        list: 분할된 텍스트 청크의 리스트.
    """
    print(f"PDF: {pdf_path} -----------------------------------")

    pdf_loader = PyPDFLoader(pdf_path)
    data_from_pdf = pdf_loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    splits = text_splitter.split_documents(data_from_pdf)

    print(f"Number of splits: {len(splits)}\n")

    return splits

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import os 

# vectorStore 설정하
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

persist_directory = '../chroma_store'

if os.path.exists(persist_directory):
    print("Loading existing Chroma store")
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
else:
    print('Creating new Chroma Store')

    vectorstore = None
    for g in glob('../data/*.pdf'):
        chunks = read_pdf_and_split_text(g)
        # 100개씩 나눠서 저장 
        for i in range(0, len(chunks), 100):
            if vectorstore is None:
                vectorstore = Chroma.from_documents(
                    documents=chunks[i:i+100],
                    embedding=embedding,
                    persist_directory=persist_directory
                )
            else:
                vectorstore.add_documents(
                    documents=chunks[i:i+100]
                )

Loading existing Chroma store


In [4]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

chunks = retriever.invoke('서울 온실가스 저감 계획')

for chunk in chunks:
    print(chunk.metadata)
    print(chunk.page_content)

{'creationdate': '2023-02-14T11:05:36+09:00', 'source': 'D:\\projects\\gpt_agent_2025_book\\chap09\\data\\2040_seoul_plan.pdf', 'moddate': '2023-02-14T18:21:42+09:00', 'creator': 'PScript5.dll Version 5.2.2', 'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'title': '', 'total_pages': 272, 'author': '', 'page': 248, 'page_label': '249'}
우려 및 지천의 복원에 대한 검토 필요
∙ 탄소중립을 계획의 주요 원칙으로 삼았으며, 부문별 전략
계획을 통해 자연과 공존하는 도시를 위한 계획 방향을 
설정
- 수변공간 활성화를 위해 단절·훼손된 녹지축과 하천축을 
복원
- 도심 속 생물다양성을 확보하여 친환경 생태기반을 마련
반영
[표 6-2] 2040 서울도시기본계획 공청회 의견 및 조치결과
2. 서울특별시의회 의견청취 결과 : 원안가결
1) 서울특별시의회 본회의 개요
 회의명 : 제314회 임시회 도시계획균형위원회
 일   자 : 2022. 9. 22. (목)
 장   소 : 서울특별시의회 도시계획균형위원회 회의실
 결   과 : 원안가결
6.1 행정절차 상 의견수렴 결과 239
2) 서울특별시의회 의견청취 및 조치결과
주요의견 조치계획 비고
∙ 6대목표에 환경부문을 추가 검토
∙ 7대 목표 중 하나로 ‘탄소중립 안
{'source': 'D:\\projects\\gpt_agent_2025_book\\chap09\\data\\2040_seoul_plan.pdf', 'author': '', 'moddate': '2023-02-14T18:21:42+09:00', 'total_pages': 272, 'creator': 'PScript5.dll Version 5.2.2', 'page_label': '177',

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

model.invoke("안녕하세요")

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DBuacH2TWV2aOmBNcgcbW4b6j6qdz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c836d-60b9-7262-920e-00a0134b1aaa-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal # 문자열 리터럴 타입을 지원하는 typing 모듈의 클래스 
from pydantic import BaseModel, Field

# Data Model
class RouteQuery(BaseModel):
    """사용자 쿼리를 가장 관련성이 높은 데이터 소스로 라우팅합니다."""

    datasource: Literal["vectorstore", "casual_talk"] = Field(
        ...,
        description="""
        사용자 질문에 따라 casual_talk 또는 vectorstore로 라우팅합니다.
        - casual_talk: 일상 대화를 위한 데이터 소스, 사용자가 일상적인 질문을 할 때 사용합니다.
        - vectorstore: 사용자 지룸에 답하기 위해 RAG로 vectorstore 검색이 필요한 경우 사용합니다.
        """
    )

In [8]:
# 특정 모델을 structured output(구조화된 출력)과 함께 사용하기 위해 설정 
structured_llm_router = model.with_structured_output(RouteQuery)

router_system="""
당신은 사용자의 질문을 vectorstore 또는 casual_talk으로 라우팅하는 전문가입니다. 
- vectorstore에는 서울, 뉴욕의 발전 계획과 관련된 문서가 포함되어 있습니다. 이 주제에 대한 질문에는 vectorstore를 사용하십시오.
- 사용자의 질문ㅇ이 일상 대화에 관련된 경우에는 casual_talk을 사용하십시오.
"""

# 시스템 메시지와 사용자의 질문을 포함하는 프롬프트 템플릿 생성 
route_prompt = ChatPromptTemplate.from_messages([
    ('system', router_system),
    ('human', "{question}")
])

# 라우터 프롬프트와 구조화된 출력 모델을 결합한 객체 
question_router = route_prompt | structured_llm_router

In [9]:
print(
    question_router.invoke({
        'question': '서울 온실가스 저감 계획은 무엇인가요?'
    })
)

print(question_router.invoke({'question': '잘 지냈어?'}))

datasource='vectorstore'
datasource='casual_talk'


In [11]:
from langchain_core.prompts import PromptTemplate

class GradeDocument(BaseModel):
    """검색된 문서가 질문과 관련성 있는지 yes 또는 no로 평가합니다."""

    binary_score: Literal["yes", 'no'] = Field(
        description="문서가 질문과 관련이 있는지 여부를 'yes' 또는 'no'로 평가합니다."
    )

structured_llm_grader = model.with_structured_output(GradeDocument)

In [12]:
grader_prompt = PromptTemplate.from_template("""
당신은 검색된 문서가 사용자 질문과 관련이 있는지 평가하는 평가자입니다 \n
문서에 사용자 질문과 관련된 키워드 또는 의미가 포함되어 있으면, 해당 문서를 관련성이 있다고 평가하십시오. \n
엄격한 테스트가 필요하지 않습니다 목표는 잘못된 검색 결과를 걸러내는 것입니다. \n
문서가 질문과 관련이 있는지 여부를 나타내기 위해 'yes' 또는 'no'로 이진 점수를 부여하십시오.
                                             
Retrived document: \n {document} \n\n
User question: {question}
""")

retrieval_grader = grader_prompt | structured_llm_grader
question = '서울시 자율주행 관련 계획'
documents = retriever.invoke(question)

for doc in documents:
    print(doc)


page_content='3.2 서울시 관련 실·국·본부 의견 195
 서울도시기본계획에 녹색교통 진흥지역 도심 확대 지정, 혼잡통행료 부과를 위한 근거 마련 필요
- 서울도시기본계획 3도심을 11월 말 녹색교통 진흥지역으로 지정 운영 발표
- 현재는 한양도성만 지정되어 운영되고 있으나 내년부터는 여의도 강남 3도심까지 확대 운영할 예정
이며 시장님 공약사항이기도 함
- 녹색교통진흥지역은 혼잡통행료를 부과하게 되는데 한양도성은 이미 남산1·3터널 통행료로 받고 
있으며 여의도 강남도 통행료 부과할 계획
- 2040에서 기능과 연계성 강화차원에서 3도심의 녹색교통진흥지역 지정에 대해 근거마련 필요
- 도심 전체를 녹색교통진흥지역으로 지정하여 혼잡통행료를 부과하는 것은 불가능하므로 지역지정 
필요
 3도심 기능강화를 위한 직결체계와 도로, 철도계획을 아울러 한꺼번에 들어가는 방안 필요
- 철도망계획, 도로건설기본계획 등을 참조
- 혼잡통행료, 자전거 CRT, 직결체계 연계해서 교통으로도 3도심의 연계성을 강화하며, 걷고싶은 
도시가 큰 틀이지만 직결체계가 있어야 함
 각 도심마다 컨셉별 녹색교통진흥지역으로 운영할 구상
- 11월말 운행제한 발표 시 말미에 녹색교통지역에 운행제한과 함께 순화번스도 들어가고 차후에 
3도심에 확대할 것을 제시 
- 한양도성은 역사도심이기도 하고 주요 도심이기 때문에 도로공간 재편을 하면서, 환경적 개념을 
더해 운행제한
- 강남은 나눔카, 공유 PM 등 공유모빌리티 체계로 집중
- 여의도는 자전거 전용도로 등 자전거 인프라가 다른 구에 비해 잘 갖춰져 있어, 국제금융지역으로 
지역 내 단거리 이동을 감안하여 자전거+PM특화지역으로 컨셉을 잡아 연구용역을 진행할 예정
 이와 같이 지역특성별 녹색교통진흥지역을 실현하기 위해 서울도시기본계획에서는 지역별 문제점
이나, 나아갈 방향을 제시 필요 
- 도심을 모두 녹색교통진흥지역으로 지정할 수 없으므로 지역선별을 할 수 밖에 없음
이나, 나아갈 방향을 제시 필요 
- 도심을 모두 녹색교

In [14]:
filtered_docs = []

for i, doc in enumerate(documents):
    print(f"Document {i+1}:")
    is_relevant = retrieval_grader.invoke({"question": question, "document": doc.page_content})
    print(is_relevant)
    print(doc.page_content[:200])
    print("=====================================")

    if is_relevant.binary_score == "yes":
        filtered_docs.append(doc)

print(f"filtered_documents: {len(filtered_docs)}")

Document 1:
binary_score='no'
3.2 서울시 관련 실·국·본부 의견 195
 서울도시기본계획에 녹색교통 진흥지역 도심 확대 지정, 혼잡통행료 부과를 위한 근거 마련 필요
- 서울도시기본계획 3도심을 11월 말 녹색교통 진흥지역으로 지정 운영 발표
- 현재는 한양도성만 지정되어 운영되고 있으나 내년부터는 여의도 강남 3도심까지 확대 운영할 예정
이며 시장님 공약사항이기도 함
- 녹색교통
Document 2:
binary_score='no'
이나, 나아갈 방향을 제시 필요 
- 도심을 모두 녹색교통진흥지역으로 지정할 수 없으므로 지역선별을 할 수 밖에 없음
- 혼잡통행료 부과 시 물류차량, 생계형 차량 등 예외 조항에 대해서는 고민이 많고 타당성을 위해 지
역적 문제점, 필요성, 연계체계가 필요하다는 내용이 서울도시기본계획에서 제시할 필요
4. 공간계획
4.1 서울시 관련 실·국·본부 의견
4
Document 3:
binary_score='yes'
140 2. 시민참여와 미래상
❚ 향후 주거환경 및 교통환경 중요 부문
 거주 생활권의 20년 후 주거환경 중 중요한 부문에 대해서는 ‘첨단기술을 기반으로 자동화 서비스를 
제공하며 편의성을 높여주는 스마트홈’이 33.4%로 가장 높았으며 다음으로 ‘잠만 자는 공간에서 
여가·상업·문화 등 다양한 기능이 복합된 다기능 주거환경’(25.3%), ‘일률적인 
Document 4:
binary_score='no'
1.1 사전자문단 회의 23
❚ 계획의 성격 및 위상
경계를 넘어서는(beyond boundary) 서울의 도시기본계획
 네트워크화가 강조될 것인데, 이 때 광역도시계획과 연결된다면 서울에 한정하지 않고 범역을 확장할 
수 있는 상황을 상정할 수 있을 것으로 생각함
공익을 추구하고 시민 보편적인 접근을 위한 도시기본계획
 부당한 개발이익이나 우발이익이 
Document 5:
binary_score='no'
194 3. 부문별 전략계획
 진정한 공론화가 되지 못하더라도 절차적 정당성을 

In [15]:
rag_generate_system="""
너는 사용자의 질문에 대해 주어진 context에 기반하여 답변하는 도시 계획 전문가이다.
주어진 context는 vectorstore에서 검색한 결과이다.
주어진 context를 기반으로 사용자의 question에 대해 답변하라.

==================================================
question: {question}
context: {context}
"""

# PromptTemplate를 생성해 question과 context를 포메팅 
rag_prompt = PromptTemplate(
    input_variables=['question', 'context'],
    template=rag_generate_system
)

# rag_chain
rag_chain = rag_prompt | model

# 사용자 질문과 검색한 문서를 입력으로 사용해 RAG를 실행 
question = "서울시 자율주행 관련 계획"

rag_chain.invoke({'question': question, 'context': filtered_docs})

AIMessage(content="서울시의 자율주행 관련 계획에 따르면, 시민들이 가장 중요하게 생각하는 교통환경 중 하나는 '완전자율주행차량의 도입으로 집 앞에서부터 목적지까지 편안하게 이동하는 교통환경'입니다. 이 항목은 조사 응답의 29.8%를 차지하며, 이는 자율주행차량이 시민들의 이동 편의성을 크게 향상시킬 수 있다는 것을 보여줍니다. \n\n따라서, 서울시는 자율주행차량을 포함한 다양한 교통수단의 융합을 통해 보다 포용적이고 편리한 교통환경을 조성하는 방향으로 계획을 세우고 있습니다. 이러한 기술적 발전은 미래의 교통체계에서 중요한 역할을 할 것으로 기대됩니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 171, 'prompt_tokens': 928, 'total_tokens': 1099, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_414ba99a04', 'id': 'chatcmpl-DBvfCzhpTFCErV3zOKruDgeEZWPrT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c83ac-5f5e-7ce3-a0e9-6692e5fc63ca-0', tool_calls=[], invalid_tool_calls=[], usage_met